Enable file persistence and internet access.

Remember that you can run the whole notebook and close the runtime without wasting resources by going to File > Save Version > Save & Run All (Double check that GPU is selected in the advanced settings).
Later, by going to 'File' > 'Version history' you can view the full logs and download the output files.

In [ ]:
!git clone https://github.com/parmigggiana/xai /kaggle/working/xai
%cd xai
!git fetch
!git reset --hard origin/main
%pip install 'monai[einops,itk,nibabel]>=1.5.0' git+https://github.com/timojl/clipseg.git

In [ ]:
from src.finetuning import FinetuneConfig, run_finetuning

DATASET_NAMES = ["CHAOS", "MMWHS"]
DOMAINS = ["CT", "MR"]
DATA_PATH = "data/"
CHECKPOINT_PATH = "checkpoints/"
OUTPUTS_PATH = "outputs/"
USE_3D = True
TRAINING_EPOCHS = {
    ("CHAOS", "CT"): 100,
    ("CHAOS", "MR"): 100,
    ("MMWHS", "CT"): 100,
    ("MMWHS", "MR"): 100,
}
BATCH_SIZE = 1
SPATIAL_SIZE = 128
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 5e-5
NUM_WORKERS = 0

SAVE_BEST = True
EARLY_STOP_PATIENCE = 3
VAL_MAX_BATCHES = None

if __name__ == "__main__":
    config = FinetuneConfig(
        dataset_names=DATASET_NAMES,
        domains=DOMAINS,
        training_epochs=TRAINING_EPOCHS,
        data_path=DATA_PATH,
        checkpoint_path=CHECKPOINT_PATH,
        outputs_path=OUTPUTS_PATH,
        use_3d=USE_3D,
        batch_size=BATCH_SIZE,
        spatial_size=SPATIAL_SIZE,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        num_workers=NUM_WORKERS,
        save_best=SAVE_BEST,
        early_stop_patience=EARLY_STOP_PATIENCE,
        val_max_batches=VAL_MAX_BATCHES,
    )
    run_finetuning(config)


In [ ]:
!zip -r /kaggle/working/checkpoints.zip /kaggle/working/xai/checkpoints